## If a tree falls in the forest...

Now that you've learned about random forests and decision trees let's do an exercise in accuracy. You know that random forests are basically a collection of decision trees. But how do the accuracies of the two models compare?

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

Hopefully out of this you'll see the power of random forests, but also their potential costs. Remember, in the real world you won't necessarily be dealing with thousands of rows. It could be millions, billions, or even more.

In [1]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
from scipy import stats
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [2]:
full_data = pd.read_csv('../data/chr/County_Health_Rankings.csv',index_col=0)

In [3]:
full_data.head()

,State,County,Life Expectancy,Age-Adjusted Mortality,Child Mortality Rate,Infant Mortality Rate,% Frequent Physical Distress,% Frequent Mental Distress,% Diabetic,HIV Prevalence Rate,% Food Insecure,% Limited Food Access,Drug Overdose Mortality Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Uninsured Adults,% Uninsured Children,% Disconnected Youth,Household Income,% Free or Reduced Lunch,Homicide Rate,Firearm Fatalities Rate,% Homeowners,% Severe Housing Cost Burden,Population,% < 18,% 65 and over,% African American,% American Indian/Alaskan Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Premature Death Years,% Fair/Poor Health,Physically Unhealthy Days,Mentally Unhealthy Days,% Low Birth Wt,% Smokers,% Obese,Food Environment Index,% Physically Inactive,% With Access to Exercise,% Excessive Drinking,% Alcohol-Impaired Deaths,Chlamydia Rate,Teen Birth Rate,% Uninsured,PCP Ratio,Dentist Ratio,MHP Ratio,Preventable Hosp. Rate,% Mammograph Screened,% Flu Vaccinated,HS Graduation Rate,% Some College,% Unemployed,% Children in Poverty,80/20 Income Ratio,% Single-Parent Households,Social Association Rate,Violent Crime Rate,Injury Death Rate,Average Daily Particulate Matter 2.5,Presence of drinking water violation,% Severe Housing Problems,% Drive Alone,% Long Commute - Drives Alone
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,76.300,439.000,53.000,8.000,13,13,14,226.000,13,12.000,10.000,20.000,36,11.000,2.000,8.000,58343.000,48.000,5.000,18.000,73,13.000,55504,23.900,15.100,19.300,0.500,1.300,0.100,2.900,74.500,1,51.300,42.000,0.159,18,4.200,4.300,8.000,19,38,7.200,31,69.000,17,29.000,341.200,27.000,9.000,2409,3084,6167,6599.000,44.000,41.000,90.000,61,3.900,19.000,4.600,25.000,12.600,272.000,74.000,11.700,No,15,86,38
1003,Alabama,Baldwin,78.600,348.000,47.000,6.000,13,13,11,164.000,12,5.000,16.000,15.000,33,14.000,3.000,8.000,56607.000,45.000,3.000,14.000,73,13.000,212628,21.800,19.900,9.000,0.800,1.200,0.100,4.600,83.000,0,51.500,42.300,0.034,18,4.100,4.200,8.000,17,31,8.000,24,72.000,17,32.000,338.800,30.000,11.000,1372,2006,1096,3833.000,45.000,45.000,86.000,66,4.000,15.000,4.500,25.000,10.700,204.000,69.000,10.300,Yes,14,85,41
1005,Alabama,Barbour,75.800,470.000,77.000,nan,16,15,18,436.000,23,11.000,nan,21.000,39,17.000,3.000,12.000,32490.000,74.000,7.000,15.000,63,14.000,25270,20.800,18.800,47.900,0.700,0.500,0.200,4.200,46.000,1,47.200,67.800,0.379,26,5.100,4.600,11.000,22,44,5.600,28,54.000,13,30.000,557.900,45.000,13.000,2597,2808,12635,4736.000,46.000,37.000,81.000,37,5.900,50.000,5.800,57.000,8.500,414.000,73.000,11.500,No,15,83,34
1007,Alabama,Bibb,73.900,564.000,112.000,15.000,13,13,15,192.000,16,3.000,22.000,25.000,38,12.000,3.000,nan,45795.000,65.000,8.000,21.000,75,9.000,22668,20.600,16.000,21.500,0.400,0.200,0.100,2.600,74.300,0,46.500,68.400,0.520,20,4.400,4.300,11.000,20,38,7.600,35,16.000,16,27.000,302.100,45.000,10.000,1742,3778,11334,5998.000,44.000,39.000,84.000,48,4.400,27.000,4.300,30.000,10.200,89.000,100.000,11.200,No,11,86,49
1009,Alabama,Blount,74.600,502.000,76.000,6.000,14,14,14,95.000,11,3.000,25.000,26.000,36,16.000,3.000,15.000,48253.000,53.000,7.000,20.000,79,8.000,58013,23.300,17.800,1.500,0.600,0.300,0.100,9.600,86.900,2,50.700,90.000,0.188,21,4.500,4.700,8.000,20,34,8.500,29,23.000,15,22.000,114.300,36.000,12.000,4439,4834,9669,4162.000,36.000,38.000,93.000,54,4.000,19.000,4.100,30.000,9.000,483.000,105.000,11.700,No,10,87,60


In [4]:
# Select medically-related fields
data = pd.DataFrame(index=full_data.index)
data['Life Expectancy'] = full_data['Life Expectancy']
data['Physical Distress Pct'] = full_data['% Frequent Physical Distress']
data['Mental Distress Pct'] = full_data['% Frequent Mental Distress']
data['Diabetic Pct'] = full_data['% Diabetic']
data['HIV Rate'] = full_data['HIV Prevalence Rate']
data['Food Insecure Pct'] = full_data['% Food Insecure']
data['Insufficient Sleep Pct'] = full_data['% Insufficient Sleep']
data['Houshold Income'] = full_data['Household Income']
data['Youth Pct'] = full_data['% < 18']
data['Elderly Pct'] = full_data['% 65 and over']
data['Female Pct'] = full_data['% Female']
data['Poor Health Pct'] = full_data['% Fair/Poor Health']
data['Physically Unhealthy Days'] = full_data['Physically Unhealthy Days']
data['Mentally Unhealthy Days'] = full_data['Mentally Unhealthy Days']
data['Low Birth Weight Pct'] = full_data['% Low Birth Wt']
data['Smoker Pct'] = full_data['% Smokers']
data['Obesity Pct'] = full_data['% Obese']
data['Inactive Pct'] = full_data['% Physically Inactive']
data['Exercise Availability Pct'] = full_data['% With Access to Exercise']
data['Excess Drinker Pct'] = full_data['% Excessive Drinking']
data['Chlamydia Rate'] = full_data['Chlamydia Rate']
data['Teen Birth Rate'] = full_data['Teen Birth Rate']
data['PCP Ratio'] = full_data['PCP Ratio']
data['Dentist Ratio'] = full_data['Dentist Ratio']
data['MHP Ratio'] = full_data['MHP Ratio']
data['Mammograph Pct'] = full_data.iloc[:,[-16]] #full_data['% Mammograph Screened']
data['Flu Vaccinated Pct'] = full_data['% Flu Vaccinated']

In [5]:
# Check whether data cleaning is needed
data.dtypes

Life Expectancy              float64
Physical Distress Pct          int64
Mental Distress Pct            int64
Diabetic Pct                   int64
HIV Rate                     float64
Food Insecure Pct              int64
Insufficient Sleep Pct         int64
Houshold Income              float64
Youth Pct                    float64
Elderly Pct                  float64
Female Pct                   float64
Poor Health Pct                int64
Physically Unhealthy Days    float64
Mentally Unhealthy Days      float64
Low Birth Weight Pct         float64
Smoker Pct                     int64
Obesity Pct                    int64
Inactive Pct                   int64
Exercise Availability Pct    float64
Excess Drinker Pct             int64
Chlamydia Rate               float64
Teen Birth Rate              float64
PCP Ratio                     object
Dentist Ratio                 object
MHP Ratio                     object
Mammograph Pct               float64
Flu Vaccinated Pct           float64
d

In [6]:
# Handle non-numeric columns
data['PCP Ratio'] = pd.to_numeric(data['PCP Ratio'], errors='coerce')
data['Dentist Ratio'] = pd.to_numeric(data['Dentist Ratio'], errors='coerce')
data['MHP Ratio'] = pd.to_numeric(data['MHP Ratio'], errors='coerce')

In [7]:
# Use above/below average life expectancy as classifier
X = data.dropna()
Y = (X['Life Expectancy'] > X['Life Expectancy'].mean())*1
X = X.drop('Life Expectancy', 1)

In [8]:
# Decision Tree model
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    random_state = 1337
)
decision_tree.fit(X, Y)

# Use grid search to find best max_features and max_depth based on accuracy
grid_values = {'max_features': range(1,15),'max_depth':range(1,8)}
grid_dt = GridSearchCV(decision_tree, param_grid = grid_values,scoring = 'accuracy',cv=5,iid=True)
grid_dt.fit(X, Y)

print('Best Decision Tree parameters: ' + str(grid_dt.best_params_))

Best Decision Tree parameters: {'max_depth': 5, 'max_features': 13}


In [9]:
# Re-run Decision Tree model using best grid search parameters
import time
start_time = time.time()

decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=5,
    max_depth=13
)
decision_tree.fit(X, Y)

cv_score = cross_val_score(decision_tree, X, Y, cv=10)

print("Decision Tree Accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))
print("Decision Tree Runtime: %s seconds" % (time.time() - start_time))

Decision Tree Accuracy: 0.77 (+/- 0.14)
Decision Tree Runtime: 0.12189459800720215 seconds


In [10]:
# Random Forest model
from sklearn import ensemble

rfc = ensemble.RandomForestClassifier()
X = pd.get_dummies(X)
X = X.dropna(axis=1)

# Use grid search to find best max_features and max_depth based on accuracy
#grid_values = {'n_estimators': range(5,25)}
grid_values = {'max_features': range(1,15),'max_depth':range(1,8)}
grid_rfc = GridSearchCV(rfc, param_grid = grid_values,scoring = 'accuracy',cv=5,iid=True)
grid_rfc.fit(X, Y)

print('Best Random Forest parameters: ' + str(grid_rfc.best_params_))

Best Random Forest parameters: {'max_depth': 6, 'max_features': 4}


In [11]:
# Re-run Random Forest model using best grid search parameters
start_time = time.time()

rfc = ensemble.RandomForestClassifier(max_features=7,max_depth=3)
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cv_score = cross_val_score(rfc, X, Y, cv=10)
print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))
print("Random Forest Runtime: %s seconds" % (time.time() - start_time))

Random Forest Accuracy: 0.83 (+/- 0.09)
Random Forest Runtime: 0.2343435287475586 seconds


The optimized random forest model has a slightly higher accuracy than the decision tree model but took almost three times as long to run. This data set is fairly small so this difference is not significant but on a larger data set, this could potentially cause computing resource constraints. 